In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [4]:
train = pd.read_csv("../CSV/train_data.csv")
test = pd.read_csv("../CSV/pred_test.csv")

print("train shape : {}, test shape : {}".format(train.shape, test.shape))

train shape : (368088, 12), test shape : (15120, 12)


In [5]:
train.head(3)

,연월일,시간,일시,year,month,day,hour,weekday,구분,구분_int,공급량,기온(°C)
0,2013-01-01,1,2013-01-01 00:00:00,2013,1,1,0,1,A,0,2497.129,-8.3
1,2013-01-01,2,2013-01-01 01:00:00,2013,1,1,1,1,A,0,2363.265,-8.5
2,2013-01-01,3,2013-01-01 02:00:00,2013,1,1,2,1,A,0,2258.505,-8.4


In [6]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

features = ['month', 'hour']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

In [7]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mse',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 37
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 2
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l2: 698693
[40]	valid_0's l2: 686950
[60]	valid_0's l2: 685714
[80]	valid_0's l2: 685538
[100]	valid_0's l2: 685487
[120]	valid_0's l2: 685473
[140]	valid_0's l2: 685465
Early stopping, best iteration is:
[142]	valid_0's l2: 685465


In [9]:
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [10]:
test_x = test[features]
preds = model.predict(test_x)
submission['공급량'] = preds
submission.to_csv("../CSV/Submission/04_two_feature_lightgbm_model.csv", index = False)

In [16]:
pred = model.predict(val_x)
a = pd.DataFrame({"y" : val_y})
b = pd.DataFrame({"pred_y" : pred})

a.reset_index(drop = True, inplace = True)
a.reset_index(inplace = True)
b.reset_index(inplace = True)

nmae(a, b)

3.686423904058824

In [12]:
import pandas as pd
import numpy as np


def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score